In [4]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import time
import random

from sklearn.svm import LinearSVC
from sklearn.kernel_approximation import RBFSampler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics.pairwise import polynomial_kernel

#plot
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from matplotlib.axes import Axes

In [2]:
X_train=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\train_x_resample3000_boundarysel_50pergroup.csv")
y_train=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\train_y_resample3000_boundarysel_50pergroup.csv")
X_val=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\scaled_validation_feature.csv")
y_val=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\validation_label.csv")

In [3]:
X_train = X_train.drop(["predict_cluster"],axis=1)
y_train = y_train.drop(["predict_cluster"],axis=1)
X_val = X_val.drop(["Unnamed: 0"],axis=1)
y_val = y_val.drop(["Unnamed: 0"],axis=1)

In [5]:
rbf_kernel=RBFSampler(gamma=1,n_components=300, random_state=42)
X_train_nonlinear=rbf_kernel.fit_transform(X_train)

In [6]:
X_train_nonlinear.shape

(248452, 300)

In [ ]:
poly = PolynomialFeatures(10)
poly.fit_transform(X_train)

In [15]:
svm_clf=LinearSVC(C=10,loss="hinge")
svm_clf.fit(X_train_nonlinear,y_train['label'])

LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0)

In [16]:
X_val_nonlinear=rbf_kernel.transform(X_val)

In [17]:
y_pred_svm=svm_clf.predict(X_val_nonlinear)

In [18]:
y_pred_svm = {"label_pred":y_pred_svm}
y_pred_svm= pd.DataFrame(data=y_pred_svm)

In [19]:
Precall,f1_score,BER,FP = cal_score (y_pred_svm,y_val['label']) 

n11:..56433..n12:..5488..n21:..4774..n22:..3332
TP:0.37777777777777777
f1 score:0.3937138130686518
FP:0.0779976146519189
BER:0.35010991843707057


In [11]:
def cal_score (y_pred,y_val):
    n11 = 0
    n12 = 0
    n21 = 0
    n22 = 0
    y_pred_array= np.array(y_pred)
    y_val_array= np.array(y_val)
    for j in range(len(y_pred_array)):
        if (y_pred_array[j]==2)&(y_val_array[j]==2):
            n22 = n22+1
        elif (y_pred_array[j]==1)&(y_val_array[j]==2):
            n12 = n12 +1
        elif (y_pred_array[j]==2)&(y_val_array[j]==1):
            n21 = n21+1
        else:
            n11 = n11+1   
    '''
    ita = np.nditer (y_pred_array)
    itb = np.nditer (y_val_array)
    while not ita.finished:
        if (ita[0]==2)&(itb[0]==2):
            n22 = n22+1
        elif (ita[0]==1)&(itb[0]==2):
            n12 = n12+1
        elif (ita[0]==2)&(itb[0]==1):
            n21 = n21+1
        else:
            n11 = n11+1
        ita.iternext()
        itb.iternext()
    '''
    try:       
        Precall = n22 / ( n12 + n22)
        Pprecision = n22 / ( n21 + n22)
        f1_score = 2 / (1/Precall + 1/Pprecision)
        FP = n21/(n21 + n11)
        FN = n12/(n12 +n22 )
        BER = 1/2*(FP+FN)   
    except Exception as ex:
        print (ex.message)
    print ("n11:.."+str(n11)+"..n12:.."+str(n12)+"..n21:.."+str(n21)+"..n22:.."+str(n22))
    print ("TP:"+str(Precall))
    print ("f1 score:" + str(f1_score))
    print ("FP:"+ str(FP))
    print ("BER:" + str(BER))
    return Precall,f1_score,BER,FP